In [2]:
import pandas as pd
import numpy as np
# Load the data
combined_df = pd.read_csv('dataset/CS-Abstract-Combined-Preprocessed-Dataset.csv')
combined_df.head()

,Paper ID,text,label,extracted_features
0,1905.02253,"This paper presents Bee$^+$, a novel insect-sc...",1,[-5.34785949e-02 1.15970001e-01 -2.49453727e-...
1,2010.14244,This paper proposes a novel approach for enabl...,1,[-1.11627234e-02 1.36427671e-01 -2.40767486e-...
2,2011.00330,This paper proposes a novel approach to overco...,1,[-1.38635410e-03 1.07046261e-01 -3.00604366e-...
3,2011.02250,This paper provides a comprehensive review of ...,1,[-4.02924139e-03 1.25577912e-01 -2.42507644e-...
4,2102.04525,The paper presents a novel loss function calle...,1,[-1.11521436e-02 1.01311266e-01 -3.34746987e-...


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt


train_df, temp_df = train_test_split(combined_df, test_size=0.3, stratify=combined_df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'])

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True, max_length=512)

In [4]:
train_dataset = TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, torch.tensor(train_df['label'].tolist()))
val_dataset = TensorDataset(val_encodings.input_ids, val_encodings.attention_mask, torch.tensor(val_df['label'].tolist()))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
# Step 4: Model Initialization
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
# Step 5: Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# Initialize lists to store metrics
train_loss_values = []
val_loss_values = []
train_accuracy_values = []
val_accuracy_values = []
train_precision_values = []
val_precision_values = []
train_f1_values = []
val_f1_values = []

# Training Loop with Evaluation
for epoch in range(3):
    # Training
    model.train()
    total_loss = 0
    train_predictions, train_true_labels = [], []
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        logits = outputs.logits
        train_predictions.extend(torch.argmax(logits, axis=1).tolist())
        train_true_labels.extend(labels.tolist())

    avg_train_loss = total_loss / len(train_loader)
    train_loss_values.append(avg_train_loss)

    train_accuracy = accuracy_score(train_true_labels, train_predictions)
    train_precision = precision_score(train_true_labels, train_predictions)
    train_f1 = f1_score(train_true_labels, train_predictions)

    train_accuracy_values.append(train_accuracy)
    train_precision_values.append(train_precision)
    train_f1_values.append(train_f1)

    # Validation
    model.eval()
    total_val_loss = 0
    val_predictions, val_true_labels = [], []
    for batch in val_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        total_val_loss += outputs.loss.item()
        logits = outputs.logits
        val_predictions.extend(torch.argmax(logits, axis=1).tolist())
        val_true_labels.extend(labels.tolist())
    
    avg_val_loss = total_val_loss / len(val_loader)
    val_loss_values.append(avg_val_loss)

    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    val_precision = precision_score(val_true_labels, val_predictions)
    val_f1 = f1_score(val_true_labels, val_predictions)

    val_accuracy_values.append(val_accuracy)
    val_precision_values.append(val_precision)
    val_f1_values.append(val_f1)
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_accuracy:.4f}, Train Precision: {train_precision:.4f}, Train F1: {train_f1:.4f}")
    print(f"Val Loss: {val_losses[-1]:.4f}, Val Accuracy: {val_accuracy:.4f}, Val Precision: {val_precision:.4f}, Val F1: {val_f1:.4f}")
    print('-----------------------------------')

In [ ]:
# Plotting metrics
epochs = range(1, 4)
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.plot(epochs, train_loss_values, label='Training Loss')
plt.plot(epochs, val_loss_values, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(epochs, train_accuracy_values, label='Training Accuracy')
plt.plot(epochs, val_accuracy_values, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(epochs, train_precision_values, label='Training Precision')
plt.plot(epochs, val_precision_values, label='Validation Precision')
plt.title('Training and Validation Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(epochs, train_f1_values, label='Training F1')
plt.plot(epochs, val_f1_values, label='Validation F1')
plt.title('Training and Validation F1 Score')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Step 6: Evaluation
model.eval()
predictions, true_labels = [], []
for batch in val_loader:
    input_ids, attention_mask, labels = [b.to(device) for b in batch]
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions.extend(torch.argmax(logits, axis=1).tolist())
    true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
auroc = roc_auc_score(true_labels, predictions)
fpr, tpr, _ = roc_curve(true_labels, predictions)
eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)

# Plotting ROC and DET Curve
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (area = {auroc})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}, AUROC: {auroc}, EER: {eer}')
